In [31]:
import pickle
import torch
import numpy as np
import networkx as nx
from tqdm import tqdm

In [7]:
def rearrange(energy_scores, candidate_position_idx, true_position_idx):
    tmp = np.array([[x==y for x in candidate_position_idx] for y in true_position_idx]).any(0)
    correct = np.where(tmp)[0]
    incorrect = np.where(~tmp)[0]
    labels = torch.cat((torch.ones(len(correct)), torch.zeros(len(incorrect)))).int()
    energy_scores = torch.cat((energy_scores[correct], energy_scores[incorrect]))
    return energy_scores, labels

import re 

def calculate_ranks_from_distance(all_distances, positive_relations):
    """
    all_distances: a np array
    positive_relations: a list of array indices

    return a list
    """
    # positive_relation_distance = all_distances[positive_relations]
    # negative_relation_distance = np.ma.array(all_distances, mask=False)
    # negative_relation_distance.mask[positive_relations] = True
    # ranks = list((negative_relation_distance < positive_relation_distance[:, np.newaxis]).sum(axis=1) + 1)
    # ranks = list((all_distances < positive_relation_distance[:, np.newaxis]).sum(axis=1) + 1)
    ranks = list(np.argsort(np.argsort(all_distances))[positive_relations]+1)
    return ranks

def obtain_ranks(outputs, targets):
    """ 
    outputs : tensor of size (batch_size, 1), required_grad = False, model predictions
    targets : tensor of size (batch_size, ), required_grad = False, labels
        Assume to be of format [1, 0, ..., 0, 1, 0, ..., 0, ..., 0]
    mode == 0: rank from distance (smaller is preferred)
    mode == 1: rank from similarity (larger is preferred)
    """
    calculate_ranks = calculate_ranks_from_distance
    all_ranks = []
    prediction = outputs.cpu().numpy().squeeze()
    label = targets.cpu().numpy()
    sep = np.array([0, 1], dtype=label.dtype)
    
    # fast way to find subarray indices in a large array, c.f. https://stackoverflow.com/questions/14890216/return-the-indexes-of-a-sub-array-in-an-array
    end_indices = [(m.start() // label.itemsize)+1 for m in re.finditer(sep.tostring(), label.tostring())]
    end_indices.append(len(label)+1)
    start_indices = [0] + end_indices[:-1]
    for start_idx, end_idx in zip(start_indices, end_indices):
        distances = prediction[start_idx: end_idx]
        labels = label[start_idx:end_idx]
        positive_relations = list(np.where(labels == 1)[0])
        ranks = calculate_ranks(distances, positive_relations)
        all_ranks.append(ranks)
    return all_ranks

In [15]:
query = 'a'

energy_scores = torch.tensor([0, 1, 3])
candidate_position_idx = [('a', 'b'), ('a', 'c'), ('a', 'g')]
# (parent, child)
node2pos = [('a', 'g'), ('a', 'b')]




In [16]:
batched_energy_scores, labels = rearrange(energy_scores, candidate_position_idx, node2pos)


In [17]:
all_ranks = obtain_ranks(batched_energy_scores, labels)

/tmp/ipykernel_1045391/610325993.py:41: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  end_indices = [(m.start() // label.itemsize)+1 for m in re.finditer(sep.tostring(), label.tostring())]


In [18]:
all_ranks

[[1, 3]]

In [19]:
import itertools

def macro_mr(all_ranks):
    macro_mr = np.array([np.array(all_rank).mean() for all_rank in all_ranks]).mean()
    return macro_mr

def micro_mr(all_ranks):
    micro_mr = np.array(list(itertools.chain(*all_ranks))).mean()
    return micro_mr

def hit_at_1(all_ranks):
    rank_positions = np.array(list(itertools.chain(*all_ranks)))
    hits = np.sum(rank_positions <= 1)
    return 1.0 * hits / len(rank_positions)

def hit_at_3(all_ranks):
    rank_positions = np.array(list(itertools.chain(*all_ranks)))
    hits = np.sum(rank_positions <= 3)
    return 1.0 * hits / len(rank_positions)

def hit_at_5(all_ranks):
    rank_positions = np.array(list(itertools.chain(*all_ranks)))
    hits = np.sum(rank_positions <= 5)
    return 1.0 * hits / len(rank_positions)

def hit_at_10(all_ranks):
    rank_positions = np.array(list(itertools.chain(*all_ranks)))
    hits = np.sum(rank_positions <= 10)
    return 1.0 * hits / len(rank_positions)

def precision_at_1(all_ranks):
    rank_positions = np.array(list(itertools.chain(*all_ranks)))
    hits = np.sum(rank_positions <= 1)
    return 1.0 * hits / len(all_ranks)

def precision_at_3(all_ranks):
    rank_positions = np.array(list(itertools.chain(*all_ranks)))
    hits = np.sum(rank_positions <= 3)
    return 1.0 * hits / (len(all_ranks)*3)

def precision_at_5(all_ranks):
    rank_positions = np.array(list(itertools.chain(*all_ranks)))
    hits = np.sum(rank_positions <= 5)
    return 1.0 * hits / (len(all_ranks)*5)

def precision_at_10(all_ranks):
    rank_positions = np.array(list(itertools.chain(*all_ranks)))
    hits = np.sum(rank_positions <= 10)
    return 1.0 * hits / (len(all_ranks)*10)

def mrr_scaled_10(all_ranks):
    """ Scaled MRR score, check eq. (2) in the PinSAGE paper: https://arxiv.org/pdf/1806.01973.pdf
    """
    rank_positions = np.array(list(itertools.chain(*all_ranks)))
    
    scaled_rank_positions = np.ceil(rank_positions / 10)
 #   print(scaled_rank_positions, (1.0 / scaled_rank_positions).mean())
    return (1.0 / scaled_rank_positions).mean()

In [22]:
test_path = '../data/MAG_CS/test_nodes.pickle'
#test_path = '../data/noun/test_hypernyms_def.pickle'
with open(test_path, 'rb') as f:
    test = pickle.load(f)


def clean(s):
    return s.split('.')[0]

new_test = []
for child, parents in test:
    temp = []
    for parent in parents:
        temp.append((clean(parent), clean(child)))
    new_test.append(temp)

In [25]:
new_test[1]

[('wireless', 'effective transmission rate'),
 ('channel', 'effective transmission rate')]

In [34]:
train = nx.read_edgelist('../data/MAG_CS/train.edgelist', create_using=nx.DiGraph, delimiter='\t')

In [40]:
ppls_path = '../data/MAG_CS/all_pairs_ppls.pickle'
with open(ppls_path, 'rb') as f:
    ppls = pickle.load(f)

In [51]:
def clean_dict(pairs):
    new_pairs = {}
    for key, val in tqdm(pairs.items()):
        term = key[0].split("(")[0].strip()

        if term == 'active or passiveThe modern Spanish verb paradigm (conjugation) has 16 distinct complete forms (tenses), i.e.)':
            term = 'spanish verbs'
        target = key[1].split(",")[0]
        new_key = (target, term)
        new_pairs[new_key] = val

    return new_pairs

ppls = clean_dict(ppls)

100%|██████████| 28242880/28242880 [00:58<00:00, 481713.96it/s]


In [48]:
new_test[1]

[('wireless', 'effective transmission rate'),
 ('channel', 'effective transmission rate')]

In [53]:
idx = 0
for k,v in ppls.items():
    
    idx += 1

    if idx >= 1000:
        print(k, v)
    if idx >= 1005:
        break

('sat', 'active or passiveThe modern Spanish verb paradigm') 2004.5543212890625
('walksat', 'active or passiveThe modern Spanish verb paradigm') 158.810302734375
('complexity index', 'active or passiveThe modern Spanish verb paradigm') 1448.5576171875
('circuit minimization for boolean functions', 'active or passiveThe modern Spanish verb paradigm') 40.19416427612305
('and inverter graph', 'active or passiveThe modern Spanish verb paradigm') 483.669921875
('post s lattice', 'active or passiveThe modern Spanish verb paradigm') 3972.873046875


In [73]:
metric_names = {
    'mrr': mrr_scaled_10,
    'p1': precision_at_1,
    'p5': precision_at_5,
    'r1': hit_at_1,
    'r5': hit_at_5
}

metrics = {}
for name in metric_names.keys():
    metrics[name] = []

missing = 0
for gold in tqdm(new_test):
    query = gold[0][1]
    
    if query == 'spanish verbs':
        query = 'active or passiveThe modern Spanish verb paradigm'
    scores = []
    potential_nodes = []
    for node in train.nodes():
        try:
            scores.append(ppls[(node, query)])
            if query == 'active or passiveThe modern Spanish verb paradigm':
                potential_nodes.append((node, 'spanish verbs'))
            else:
                potential_nodes.append((node, query))

        except KeyError:
            missing += 1
    
    scores = torch.tensor(scores)
    batched_energy_scores, labels = rearrange(scores, potential_nodes, gold)
    all_ranks = obtain_ranks(batched_energy_scores, labels)

    for name, func in metric_names.items():
        cur_metric = np.nan_to_num(func(all_ranks))
        metrics[name].append(cur_metric)

  0%|          | 0/1000 [00:00<?, ?it/s]/tmp/ipykernel_1045391/610325993.py:41: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  end_indices = [(m.start() // label.itemsize)+1 for m in re.finditer(sep.tostring(), label.tostring())]
/tmp/ipykernel_1045391/1455042432.py:58: RuntimeWarning: Mean of empty slice.
  return (1.0 / scaled_rank_positions).mean()
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/tmp/ipykernel_1045391/1455042432.py:14: RuntimeWarning: invalid value encountered in scalar divide
  return 1.0 * hits / len(rank_positions)
/tmp/ipykernel_1045391/1455042432.py:24: RuntimeWarning: invalid value encountered in scalar divide
  return 1.0 * hits / len(rank_positions)
100%|██████████| 1000/1000 [01:23<00:00, 11.93it/s]


In [74]:
for name, v in metrics.items():
    print(name, np.mean(v))

mrr 0.34832617452054476
p1 0.174
p5 0.0788
r1 0.12655396825396825
r5 0.2419690476190476


In [76]:
new_test[1]

[('wireless', 'effective transmission rate'),
 ('channel', 'effective transmission rate')]

In [77]:
ppls[('wireless', 'effective transmission rate')]

86.04059600830078

In [ ]:
ppls[('wireless', 'effective transmission rate')]

In [75]:
metrics

{'mrr': [0.3333333333333333,
  0.0015774567489242118,
  0.0,
  0.008403361344537815,
  1.0,
  0.0038022813688212928,
  0.007692307692307693,
  0.006814019314019314,
  0.015873015873015872,
  0.002079002079002079,
  0.006993006993006993,
  0.5069444444444444,
  0.0,
  0.0,
  0.0,
  1.0,
  0.003484320557491289,
  0.02503082139004469,
  0.011904761904761904,
  0.7504854368932039,
  1.0,
  0.0,
  0.058823529411764705,
  0.0,
  0.6962962962962963,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.0,
  0.06380890052356021,
  1.0,
  0.0,
  0.5,
  0.038461538461538464,
  1.0,
  1.0,
  0.006756756756756757,
  1.0,
  1.0,
  0.00273224043715847,
  0.10574712643678161,
  0.002806585590290731,
  0.5040322580645161,
  0.125,
  0.0,
  0.03428543134425487,
  0.004032258064516129,
  1.0,
  1.0,
  0.09090909090909091,
  0.5164473684210527,
  0.004366812227074236,
  1.0,
  0.0,
  0.002288329519450801,
  1.0,
  0.8333333333333334,
  0.5,
  1.0,
  0.07692307692307693,
  1.0,
  0.012424698795180721,
  1.0,
  0.

In [79]:
query = new_test[1][0][1]

if query == 'spanish verbs':
    query = 'active or passiveThe modern Spanish verb paradigm'
scores = []
potential_nodes = []
for node in train.nodes():
    try:
        scores.append(ppls[(node, query)])
        if query == 'active or passiveThe modern Spanish verb paradigm':
            potential_nodes.append((node, 'spanish verbs'))
        else:
            potential_nodes.append((node, query))

    except KeyError:
        missing += 1

In [83]:
scored = list(zip(potential_nodes, scores))


In [85]:
sorted(scored, key=lambda x: x[1])

[(('throughput', 'effective transmission rate'), 1.722144603729248),
 (('bit error rate', 'effective transmission rate'), 1.8903053998947144),
 (('wavelength division multiplexing', 'effective transmission rate'),
  1.9561984539031982),
 (('carrier sense multiple access with collision avoidance',
   'effective transmission rate'),
  2.1193759441375732),
 (('fiber optic splitter', 'effective transmission rate'), 2.371079921722412),
 (('adaptive quality of service multi hop routing',
   'effective transmission rate'),
  2.4072701930999756),
 (('error detection and correction', 'effective transmission rate'),
  2.569766044616699),
 (('data transmission', 'effective transmission rate'), 2.627894401550293),
 (('bandwidth', 'effective transmission rate'), 2.7350857257843018),
 (('rate monotonic scheduling', 'effective transmission rate'),
  2.746619462966919),
 (('dispersion shifted fiber', 'effective transmission rate'),
  2.861107349395752),
 (('interferometric synthetic aperture radar', '

In [ ]:
for idx in range(len(new_test)):
    #hyps = get_hypernyms(new_pred[idx])
    hyps = new_pred[idx] + [', ']
   # print(hyps)
    gold = new_test[idx]

    child = gold[0][1]
    new_hyps = [(hyp, child) for hyp in hyps]
    scores = torch.arange(len(new_hyps))

    batched_energy_scores, labels = rearrange(scores, new_hyps, gold)

    all_ranks = obtain_ranks(batched_energy_scores, labels)
    for name, func in metric_names.items():
        cur_metric = np.nan_to_num(func(all_ranks))
        metrics[name].append(cur_metric)
